# Resource Estimation of QAOA

> Note: 
> * The charts need to be rendered for every run unlike other outputs. So the charts from ```EstimatesOverview``` won't be visible when you open the notebook, but will be visible when you run the notebook.

In [1]:
# General imports
import numpy as np
import matplotlib.pyplot as plt

# Qsharp
import qsharp
from qsharp_widgets import EstimatesOverview
from qsharp.estimator import EstimatorParams, QubitParams, QECScheme

In [2]:
%%qsharp
// This is the Q# snippet used to create and simulate the QAOA circuit. 
namespace qaoa_note{

    open Microsoft.Quantum.Measurement;

    // Function for getting flat index
    operation flat_index(n: Int, i: Int, j: Int): Int{
        return n*i + j
    }
    // Cost Hamiltonian
    operation cost_unitary(qubits: Qubit[], gamma: Double, quadratics: Double[], linears: Double[]): Unit{
        
        let n_qubits = Length(linears);
        mutable quad_sum : Double = 0.0;

        // RZ Gates
        for qubit in 0..n_qubits-1{
            set quad_sum = 0.0;
            for quad_qubit in 0..n_qubits-1{
                set quad_sum += quadratics[flat_index(n_qubits,qubit,quad_qubit)];
            }
            Rz(0.5 * (linears[qubit] + quad_sum) * gamma, qubits[qubit])
        }
        // RZZ Gates
        for i in 0..n_qubits-1{
            for j in i+1..n_qubits-1{                
                Rzz(0.25 * quadratics[flat_index(n_qubits,i,j)] * gamma, qubits[i], qubits[j])
            }
        }
    }

    // Mixer Hamiltonian
    operation mixer_unitary(qubits: Qubit[], beta: Double) : Unit{
        for qubit in qubits{
            Rx(2.0 * beta,qubit);
        }
    }

    // Function to create the QAOA circuit.
    operation circuit(NQubits: Int, Layers: Int, gammas: Double[], betas: Double[], quadratics: Double[], linears: Double[]) : Int {

        use q = Qubit[NQubits]; 
        mutable integer_result = 0;
        
        // State Preparation |+>
        ApplyToEachA(H,q);

        for layer in 0..Layers-1{
            cost_unitary(q, gammas[layer], quadratics, linears);
            mixer_unitary(q, betas[layer]);
        }
        // Return the bitstring as an integer.
        return MeasureInteger(q);
    }
}


## Using the Resourse Estimator

In [3]:
def arr_to_str(a):
    """
    Converts a given array to a string representation.

    Args:
        a (list): The array to be converted.

    Returns:
        str: The string representation of the array.

    Example:
        >>> arr_to_str([1, 2, 3])
        '[1,2,3]'
    """
    string =''
    for i in a:
        string += str(i) + ","
    return '[' + string[:-1] + ']'

def generate_qsharp_string(n_qubits: int = 3,layers: int = 3) -> str:
    
    # Preparing inputs for the Q# function.
    quadratics = np.array([1.1] * n_qubits**2)
    linears = np.array([1.2] * n_qubits)

    gammas = np.random.rand(layers)
    betas = np.random.rand(layers)

    input_str = f"{n_qubits},{layers},{arr_to_str(gammas)},{arr_to_str(betas)},{arr_to_str(quadratics)},{arr_to_str(linears)}"

    qsharp_string = f"qaoa_note.circuit({input_str})"

    return qsharp_string

In [4]:
input_str = generate_qsharp_string(3,3)

In [5]:
result = qsharp.estimate(input_str, params={"errorBudget": 0.01, "qubitParams": {"name": "qubit_maj_ns_e6"}, "qecScheme": {"name": "floquet_code"},"estimateType": "frontier", "constraints": {"logicalDepthFactor": 4}})

estimates_overview = EstimatesOverview(result)
estimates_overview

EstimatesOverview(estimates={'status': 'success', 'jobParams': {'qecScheme': {'name': 'floquet_code', 'errorCo…

In [6]:
labels = ["Gate-based µs, 10⁻³", "Gate-based µs, 10⁻⁴", "Gate-based ns, 10⁻³", "Gate-based ns, 10⁻⁴", "F-Majorana ns, 10⁻⁴", "F-Majorana ns, 10⁻⁶", "S-Majorana ns, 10⁻⁴", "S-Majorana ns, 10⁻⁶"]

params = EstimatorParams(num_items=8)
params.error_budget = 0.333
params.items[0].qubit_params.name = QubitParams.GATE_US_E3
params.items[0].qubit_params.estimateType = "frontier"
params.items[1].qubit_params.name = QubitParams.GATE_US_E4
params.items[1].qubit_params.estimateType = "frontier"
params.items[2].qubit_params.name = QubitParams.GATE_NS_E3
params.items[2].qubit_params.estimateType = "frontier"
params.items[3].qubit_params.name = QubitParams.GATE_NS_E4
params.items[3].qubit_params.estimateType = "frontier"
params.items[4].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[4].qubit_params.estimateType = "frontier"
params.items[4].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[5].qubit_params.name = QubitParams.MAJ_NS_E6
params.items[5].qubit_params.estimateType = "frontier"
params.items[5].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[6].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[6].qubit_params.estimateType = "frontier"
params.items[6].qec_scheme.name = QECScheme.SURFACE_CODE
params.items[7].qubit_params.name = "qubit_maj_ns_e6"
params.items[7].qec_scheme.name = QECScheme.SURFACE_CODE
params.items[7].qubit_params.estimateType = "frontier"


# results = qsharp.estimate(f"qaoa_note.circuit({input_str})", params=params).summary_data_frame(labels=labels)
# display(results[:])

result = qsharp.estimate(input_str, params=params)

estimates_overview = EstimatesOverview(result,runNames=labels)
estimates_overview

EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…

In [7]:
result = qsharp.estimate(
    input_str, 
    params=[
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_gate_ns_e3" },
            "qecScheme": { "name": "surface_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_gate_ns_e4" },
            "qecScheme": { "name": "surface_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_gate_us_e3" },
            "qecScheme": { "name": "surface_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_gate_us_e4" },
            "qecScheme": { "name": "surface_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_maj_ns_e4" },
            "qecScheme": { "name": "surface_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_maj_ns_e4" },
            "qecScheme": { "name": "floquet_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_maj_ns_e6" },
            "qecScheme": { "name": "surface_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        },
        {
            "error_budget": 0.333,
            "qubitParams": { "name": "qubit_maj_ns_e6" },
            "qecScheme": { "name": "floquet_code" },
            "estimateType": "frontier", # Pareto frontier estimation
        }
    ]
)

runNames = [
    "Gate-based µs, 10⁻³", "Gate-based µs, 10⁻⁴",
    "Gate-based ns, 10⁻³", "Gate-based ns, 10⁻⁴",
    "S-Majorana ns, 10⁻⁴", "F-Majorana ns, 10⁻⁴",
    "S-Majorana ns, 10⁻⁶", "F-Majorana ns, 10⁻⁶"
]

EstimatesOverview(result, runNames=runNames)


EstimatesOverview(estimates={0: {'status': 'success', 'jobParams': {'qecScheme': {'name': 'surface_code', 'err…